# Generating shakespeare-like texts using RNN 

## Importing Libraries

In [62]:
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

## Importing the data

In [4]:
file_path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [5]:
text = open(file_path, 'rb').read().decode(encoding = 'utf-8').lower()

In [8]:
len(text)

1115394

In [9]:
characters = sorted(set(text))

In [49]:
char_to_index = dict((character, index) for (index, character) in enumerate(characters)) 
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'k': 23,
 'l': 24,
 'm': 25,
 'n': 26,
 'o': 27,
 'p': 28,
 'q': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'x': 36,
 'y': 37,
 'z': 38}

In [50]:
index_to_char = dict((index, character) for (index, character) in enumerate(characters))
index_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'a',
 14: 'b',
 15: 'c',
 16: 'd',
 17: 'e',
 18: 'f',
 19: 'g',
 20: 'h',
 21: 'i',
 22: 'j',
 23: 'k',
 24: 'l',
 25: 'm',
 26: 'n',
 27: 'o',
 28: 'p',
 29: 'q',
 30: 'r',
 31: 's',
 32: 't',
 33: 'u',
 34: 'v',
 35: 'w',
 36: 'x',
 37: 'y',
 38: 'z'}

In [51]:
sentences = []
next_characters = []

sequence_length = 50
step_size = 2

for i in range(0, len(text) - sequence_length, step_size):
    sentences.append(text[i:i+sequence_length])
    next_characters.append(text[i+sequence_length])

In [52]:
sentences[0]

'first citizen:\nbefore we proceed any further, hear'

In [53]:
len(sentences)

557672

In [54]:
x = np.zeros((len(sentences), sequence_length, len(characters)), dtype = bool)
y = np.zeros((len(sentences), len(characters)), dtype = bool)

In [55]:
sentences[0:10]

['first citizen:\nbefore we proceed any further, hear',
 'rst citizen:\nbefore we proceed any further, hear m',
 't citizen:\nbefore we proceed any further, hear me ',
 'citizen:\nbefore we proceed any further, hear me sp',
 'tizen:\nbefore we proceed any further, hear me spea',
 'zen:\nbefore we proceed any further, hear me speak.',
 'n:\nbefore we proceed any further, hear me speak.\n\n',
 '\nbefore we proceed any further, hear me speak.\n\nal',
 'efore we proceed any further, hear me speak.\n\nall:',
 'ore we proceed any further, hear me speak.\n\nall:\ns']

In [56]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i,t,char_to_index[character]] = 1
        
    y[i,char_to_index[next_characters[i]]] = 1 

## Building the Model

In [60]:
model = Sequential()
model.add(LSTM(128, input_shape = (sequence_length, len(characters))))
model.add(Dense(len(characters), activation = 'softmax'))

In [61]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               86016     
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(optimizer = RMSprop(learning_rate = 0.01),
              loss = 'categorical_crossentropy')

In [68]:
model.fit(x, y, batch_size = 256, epochs = 4)

Epoch 1/4
2179/2179 [==============================] - 170s 77ms/step - loss: 1.8378
Epoch 2/4
2179/2179 [==============================] - 166s 76ms/step - loss: 1.5321
Epoch 3/4
2179/2179 [==============================] - 149s 68ms/step - loss: 1.4653
Epoch 4/4
2179/2179 [==============================] - 149s 68ms/step - loss: 1.4307


In [69]:
model.save('shakespeare_text_generator.model')

INFO:tensorflow:Assets written to: shakespeare_text_generator.model\assets


INFO:tensorflow:Assets written to: shakespeare_text_generator.model\assets


In [70]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Making the text generation function

In [71]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - sequence_length - 1)
    generated_text = ''
    sentence = text[start_index: start_index + sequence_length]
    generated_text = generated_text + sentence 
    for i in range(length):
        x = np.zeros((1, sequence_length, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1
        
        predictions = model.predict(x, verbose = 0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]
        
        generated_text = generated_text + next_character
        sentence = sentence[1:] + next_character
    
    return generated_text

## Printing some examples

In [74]:
print(generate_text(500, 0.25))

and aloof, and so i did:
anon comes one with light to the persuition
with the sons in the course of the sund to the terms
and the course and the lancess the senserse,
and the propheses and the love the tent to me,
and the course of the soul the senteness of my son,
the prove of the sabse to the heart the servers
the lord is the soul that she have the dead.

lucio:
our good mance of my honour to the father to the
with the souls the traitor be so the coursely dead,
and the thing of the course with the words,
and the subjects to the propheses to h


In [75]:
print(generate_text(500, 0.5))

et of angelo's request,
and fit his mind to death, and all his house,
where i have there is the his carry with him,
or warwick and the worth the sen'er while there is
the richmond the vault of souls in the counter of
the soul so more with the selfsity: which i may dead.

virgilia:
i shall dead and denielians here and the
then the soul the soul of the subjects send my face
to the could indeed by the father to the lord,
the prince of such in so is done to off.

somerset:
i will such a south of the good head?

baptista:
i say, sir, i was to be dou


In [76]:
print(generate_text(500, 0.75))

ishonoured by this new marriage.

king edward iv:
can and thee were me the theep of souls,
i shall shall not hear me is courice, i see them.
the liver come to me the unto thee, do your life.
but make to had not be sweet hourainers
at the name comes with them, and so provosed.
first the pride rebeleans to-neak with her.

lady anne:
no.

capulet:
stand past of his courrce.

friar pley:
whit help, here here the now, who be the most more for
right dead dishill of a good daulth was the day
of the fool, our lucesser were to be strokes,
with where i c
